<a href="https://colab.research.google.com/github/jsuj1th/Colab/blob/main/33500274_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### CSCE 670 :: Information Storage & Retrieval :: Texas A&M University :: Spring 2025


# Homework 1:  OASIS Search Engine, The Beginning

### 100 points [4% of your final grade]

### Due: February 5 (Wednesday) by 11:59pm

*Goals of this homework:* In this homework you will get first hand experience building a text-based mini search engine + explore some LLM capabilities.

*Submission instructions (Canvas):* To submit your homework, rename this notebook as `UIN_hw1.ipynb`. For example, my homework submission would be something like `555001234_hw1.ipynb`. Submit this notebook via Canvas (look for the homework 1 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Canvas, search StackOverflow, even use ChatGPT. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. See the course syllabus for details.

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Dataset: Enron Email Dataset

We are providing you with a small collection of emails from the Enron Email Dataset. The Enron Email Dataset was collected and prepared by the CALO Project (A Cognitive Assistant that Learns and Organizes). It contains data from about 150 users, mostly senior management of Enron. The full corpus contains a total of about 0.5M messages (https://www.cs.cmu.edu/~enron/). For this homework, we will use a small subset of the data. The subset contains 814 emails extracted from the `_sent_mail` of Arnold-J. We have zipped the 814 files (each file contains the information of an email). The zipped file is available on Canvas as `enron_814.zip`. The subset we provide is about 1.1MB. You should treat each email as a unique document to be indexed by your system. You can download the data from Canvas to your local filesystem. We're going to use these emails as the basis of OASIS, our Open Access Searchable Information System!


Below is an example of one email.

```text
Message-ID: <33025919.1075857594206.JavaMail.evans@thyme>
Date: Wed, 13 Dec 2000 13:09:00 -0800 (PST)
From: john.arnold@enron.com
To: slafontaine@globalp.com
Subject: re:spreads
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: John Arnold
X-To: slafontaine@globalp.com @ ENRON
X-cc:
X-bcc:
X-Folder: \John_Arnold_Dec2000\Notes Folders\'sent mail
X-Origin: Arnold-J
X-FileName: Jarnold.nsf

saw a lot of the bulls sell summer against length in front to mitigate
margins/absolute position limits/var.  as these guys are taking off the
front, they are also buying back summer.  el paso large buyer of next winter
today taking off spreads.  certainly a reason why the spreads were so strong
on the way up and such a piece now.   really the only one left with any risk
premium built in is h/j now.   it was trading equivalent of 180 on access,
down 40+ from this morning.  certainly if we are entering a period of bearish
to neutral trade, h/j will get whacked.  certainly understand the arguments
for h/j.  if h settles $20, that spread is probably worth $10.  H 20 call was
trading for 55 on monday.  today it was 10/17.  the market's view of
probability of h going crazy has certainly changed in past 48 hours and that
has to be reflected in h/j.




slafontaine@globalp.com on 12/13/2000 04:15:51 PM
To: slafontaine@globalp.com
cc: John.Arnold@enron.com
Subject: re:spreads



mkt getting a little more bearish the back of winter i think-if we get another
cold blast jan/feb mite move out. with oil moving down and march closer flat
px
wide to jan im not so bearish these sprds now-less bullish march april as
well.
```

# Part 1: Read and Parse the Email Data (20 points)

Recall how we handled file input in Homework 0? Well, here, our goal is to read the emails so that we can begin to tokenize them later. For this step, you should read the dataset and print the emails. Note that our dataset contains multiple files. You will need to write Python code to read from these files, and then build a list to store the documents. Each item in the list should be a dictionary containing the `Document-ID` as the key, and email content as the value. You can discard all the supplementary information of the email, e.g., `Date`, `From`, `To`, `Subject`, etc.

A document should look like:

```text
{'Document-ID': '33025919.1075857594206',
'content': 'saw a lot of the bulls sell summer against length in front to mitigate
margins/absolute position limits/var.  as these guys are taking off the
front, they are also buying back summer.  el paso large buyer of next winter
today taking off spreads.  certainly a reason why the spreads were so strong
on the way up and such a piece now.   really the only one left with any risk
premium built in is h/j now.   it was trading equivalent of 180 on access,
down 40+ from this morning.  certainly if we are entering a period of bearish
to neutral trade, h/j will get whacked.  certainly understand the arguments
for h/j.  if h settles $20, that spread is probably worth $10.  H 20 call was
trading for 55 on monday.  today it was 10/17.  the market's view of
probability of h going crazy has certainly changed in past 48 hours and that
has to be reflected in h/j.




slafontaine@globalp.com on 12/13/2000 04:15:51 PM
To: slafontaine@globalp.com
cc: John.Arnold@enron.com
Subject: re:spreads



mkt getting a little more bearish the back of winter i think-if we get another
cold blast jan/feb mite move out. with oil moving down and march closer flat
px
wide to jan im not so bearish these sprds now-less bullish march april as
well.'
}
```

For this homework, you should treat the email content as a document and the Message-ID as the document ID.

## Print the first two documents (5 points)

Your output should look like this:

DocumentID Document

33025919.1075857594206 saw a lot of the bulls sell summer ......

...


In [6]:
!apt-get install p7zip-full
!7z x enron_814.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1036170 bytes (1012 KiB)

Extracting archive: enron_814.zip
--
Path = enron_814.zip
Type = zip
Physical Size = 1036170

  0%    
Would you like to replace the existing file:
  Path:     ./__MACOSX/._enron_814
  Size:     176 bytes (1 KiB)
  Modified: 2025-01-17 20:21:10
with the file from archive:
  Path:     __MACOSX/._enron_814
  Size:     176 bytes (1 KiB)
  Modified: 2025-01-17 20:21:10
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? A

  0% 1 - __MACOSX/._enron_814

In [33]:
# your code here
# please print out the first 2 docs
import re, os, json
from itertools import islice
Mails=[]
doc={}

for file in os.listdir("enron_814"):
  with open(os.path.join("enron_814",file), 'r', encoding='utf-8') as f:
    mail= f.read()
    match = re.search(r"(\d+\.\d+)", mail)
    if match:
      document_id = match.group(1)
      # print(document_id)
      # doc["Document-ID"]=document_id
      # doc["content"]=contents
      content=mail.split("\n")[16:]
      content_str=""
      for i in content:
        content_str+=i
      doc[document_id]=content_str
      # Mails.append(doc)

# print(Mails)
# print(json.dumps(doc, indent=4, ensure_ascii=False))
# print(len(Mails))
# print(doc)
# print(content)
# print(mail)
for key, value in islice(doc.items(), 2):
    print(key, value)
# print(contents.split("\n")[16:])



19325260.1075857596087 i heard he's interviewing to be Shankman's right hand manJennifer Burns10/20/2000 02:12 PMTo: John Arnold/HOU/ECT@ECTcc:  Subject: I saw Bill Perkins today, he was on 33.  I heard someone call my name and I was like hey Bill.  Weird huh?
33025919.1075857594206 saw a lot of the bulls sell summer against length in front to mitigate margins/absolute position limits/var.  as these guys are taking off the front, they are also buying back summer.  el paso large buyer of next winter today taking off spreads.  certainly a reason why the spreads were so strong on the way up and such a piece now.   really the only one left with any risk premium built in is h/j now.   it was trading equivalent of 180 on access, down 40+ from this morning.  certainly if we are entering a period of bearish to neutral trade, h/j will get whacked.  certainly understand the arguments for h/j.  if h settles $20, that spread is probably worth $10.  H 20 call was trading for 55 on monday.  today it

Now that you can read the documents, let's move on to tokenization. We are going to simplify things for you:
1. You should **lowercase** all words.
2. Replace line breaks (e.g., \n, \n\n), punctuations, dashes and splash (e.g., -, /) and special characters (\u2019, \u2005, etc.) with empty space (" ").
3. Tokenize the documents by splitting on whitespace.
4. Then only keep words that have a-zA-Z in them.

In [34]:
# your code here

# def custom_tokenize(text):
#     text = text.lower()
#     text = re.sub(r'[^\w\s]', ' ', text)
#     tokens = text.split()
#     tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]
#     return tokens

def custom_tokenize(text):
    # Lowercase the text
    text = text.lower()

    # Replace line breaks, punctuation, dashes, slashes, and special characters with space
    text = re.sub(r'[\n\r\t.,!?;:\-\/\u2019\u2005]', ' ', text)

    # Tokenize by splitting on whitespace
    tokens = text.split()

    # Keep only words containing at least one letter (a-zA-Z)
    tokens = [word for word in tokens if re.search(r'[a-zA-Z]', word)]

    return tokens
tokens=[]
print(doc)
for id,mail in doc.items():
  tks=custom_tokenize(mail)
  for tk in tks:
    tokens.append(tk)
print(len(set(tokens)))

print(doc)


{'19325260.1075857596087': "i heard he's interviewing to be Shankman's right hand manJennifer Burns10/20/2000 02:12 PMTo: John Arnold/HOU/ECT@ECTcc:  Subject: I saw Bill Perkins today, he was on 33.  I heard someone call my name and I was like hey Bill.  Weird huh?", '33025919.1075857594206': "saw a lot of the bulls sell summer against length in front to mitigate margins/absolute position limits/var.  as these guys are taking off the front, they are also buying back summer.  el paso large buyer of next winter today taking off spreads.  certainly a reason why the spreads were so strong on the way up and such a piece now.   really the only one left with any risk premium built in is h/j now.   it was trading equivalent of 180 on access, down 40+ from this morning.  certainly if we are entering a period of bearish to neutral trade, h/j will get whacked.  certainly understand the arguments for h/j.  if h settles $20, that spread is probably worth $10.  H 20 call was trading for 55 on monday

## Print the first two documents after tokenizing (5 points)

Once you have your parser working, you should print the first two documents (documentID and tokens).

Your output should look like this:

* DocumentID Tokens



In [32]:
# your code and output here
for id,mail in doc.items():
  print(mail)
  doc[id]=custom_tokenize(mail)
# print(json.dumps(Mails[:2], indent=4, ensure_ascii=False))
# print(Mails[:2])
print(doc)

['i', 'heard', "he's", 'interviewing', 'to', 'be', "shankman's", 'right', 'hand', 'manjennifer', 'burns10', 'pmto', 'john', 'arnold', 'hou', 'ect@ectcc', 'subject', 'i', 'saw', 'bill', 'perkins', 'today', 'he', 'was', 'on', 'i', 'heard', 'someone', 'call', 'my', 'name', 'and', 'i', 'was', 'like', 'hey', 'bill', 'weird', 'huh']


AttributeError: 'list' object has no attribute 'lower'

## Dictionary Size (5 points)

Next you should report the size of your dictionary, that is, how many unique tokens among all the documents.




In [10]:
print("No.of Unique Tokens:",len(set(tokens)))


No.of Unique Tokens: 13566


## Top-20 Words (5 points)

Finally, you should print a list of the top-20 most popular words by counting among all documents.


Your output should look like this:

* Rank. Token, Count

1. awesome, 20
2. cool, 15
3. ...

In [11]:
from collections import Counter
word_counts = Counter(tokens)
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
sorted_word_counts[:20]


[('the', 3760),
 ('to', 2821),
 ('a', 1761),
 ('and', 1676),
 ('of', 1623),
 ('you', 1586),
 ('on', 1520),
 ('in', 1384),
 ('hou', 1345),
 ('i', 1308),
 ('john', 1196),
 ('is', 1146),
 ('for', 1091),
 ('subject', 839),
 ('ect@ect', 796),
 ('that', 765),
 ('it', 729),
 ('be', 692),
 ('from', 640),
 ('this', 636)]

# Part 2: Boolean Retrieval (30 points)

In this part you will build an inverted index to support Boolean retrieval. You should use the tokenization strategy from above.

We require your index to support AND, OR, NOT queries.

Search for the queries below using your index and print out matching documents (for each query, print out 5 matching documents):
* buyer
* margins AND limits
* winter OR summer
* buyers AND risk AND NOT crazy
* never OR know

Recall, that you should apply the exact same pre-processing strategies to the query as we do to the documents.

The output should like this:
* DocumentID Document

To make our life easier, please output the DocumentIDs in lexicographic order.

In [12]:
# build the index here
# add cells as needed to organize your code
def retrieve(query):



SyntaxError: incomplete input (<ipython-input-12-a7b2f1ce9e7a>, line 4)

## Running the five queries (4 points each, 20 points in total)

In [ ]:
# search for the input using your index and print out ids of matching documents.

Now show the results for the query: `buyer`

In [ ]:
# your code here

*Now* show the results for the query: `margins AND limits`

---




In [ ]:
# your code here

Now show the results for the query: `winter OR summer`

In [ ]:
# your code here

Now show the results for the query: `buyers AND risk AND NOT crazy`

In [ ]:
# your code here

Now show the results for the query: `never OR know`

In [ ]:
# your code here

## Observations (10 points)
Does your boolean search engine find relevant documents for these queries? As in, would our customers be happy if we shipped this retrieval engine? Why or why not?

What is the impact of the pre-processing options? Do they impact your search quality?

*your discussion here*

# Part 3: Ranking (35 points)

For the third part, you will add ranking to your search system. Given a search query, our goal is to retrieve the top-5 most relevant emails by assigning a score to each document.

We will explore two ranking methods, each offering a different approach to scoring and ranking documents:


### Ranking method A: Ranking with vector space model with TF-IDF (15 points)

**Cosine:** You should use cosine as your scoring function.

**TFIDF:** For the **document vectors**, use the standard TF-IDF scores introduced in class. For the **query vector**, use **simple weights (the raw term frequency)**. For example:
* query: never $\rightarrow$ (1)
* query: never know $\rightarrow$ (1, 1)

**Query:**  `trade`

**Output:**
You should output the top-5 results plus the cosine score of each of these documents.  

The output should be like this:

Rank Scores DocumentID Document

---

You can additionally assume that your queries will contain at most three words. Be sure to normalize your vectors as part of the cosine calculation!


In [ ]:
# your code here


# print out the top-5 retrieved emails

### Ranking method B: Ranking with BM25 (15 points)
Finally, let's try the BM25 approach for ranking. Refer to https://en.wikipedia.org/wiki/Okapi_BM25 for the specific formula. You could choose k_1 = 1.2 and b = 0.75 but feel free to try other options.

**Query:**  `gas floor`

**Output:**
You should output the top-5 results plus the BM25 score of each of these documents.  

The output should be like this:

Rank Scores DocumentID Document

In [ ]:
# your code here


# print out the top-5 retrieved emails

## Observations (5 points)
What do you observe? Are there key differences between the two ranking approaches? Briefly discuss in bullet points.


* Your observations:
* Differences:

# Part 4: Cool LLM RAG Extension (15 points)

Finally, we give you an opportunity to explore using OASIS for Retrieval-Augmented Generation (RAG) with LLMs.
Here, the task is to retrieve the top-5 emails for a query you like. You will then pass the retrieved email content along with your question to the LLM and let it answer your question. Specifically, we want you:

* Query the LLM directly with your question.
* Retrieve the top-5 emails based on your query and pass them to the LLM along with your question.
* How is the RAG output different from the non-RAG output? Does retrieval help the LLM better answer your question?

We recommend using Gemini following the [instructions](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Prompting.ipynb) here.

Hint: Take a close look at the dataset and pick a specific, relevant query where retrieval can enhance the LLM’s response.

*What You Will Submit*
- Your query.
- The top-5 retrieved emails (including Document ID and ranking score).
- The LLM's response without retrieval.
- The LLM's response with retrieval (RAG).
- A brief analysis comparing both outputs.

We will grade this last part according to correctness, effort, and creativity.

## Step 1: Query the LLM Directly (Without Retrieval) (5 points)

In [ ]:
# your code here to prompt LLM

## Step 2: Query the LLM with Retrieved Emails (RAG) (5 points)

In [ ]:
# print out the LLM response without the email content


# print out the LLM response with the email content (RAG)



## Discussion (5 points)

In this section, reflect on the performance of different ranking methods and the impact of retrieval on LLM responses. Consider:

- How did retrieval affect the LLM’s response? Did it improve factual accuracy or relevance?
- Were there cases where retrieval hurt performance (e.g., irrelevant documents, redundancy)?
- Any ideas for improving the ranking or retrieval process?

Keep your discussion *concise* and *insightful*, focusing on key takeaways from your experiments. Please answer in bullet points.

*your discussion here*

* point 1
* point 2
* ...

# Collaboration Declarations

*You should fill out your collaboration declarations here.*

https://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python

ChatGPT:
- i want to no. of occurences of each element in a list